In [1]:
import pandas
pandas.options.mode.chained_assignment = None
import sys
import numpy as np
from globsML.utils.preprocessing import rescale_data, select_galaxies, create_data_dict, replace_NaN
from globsML.models.skmodels import StandardClassifier

data_path = '../../../data/ACS_sources_original.csv'
data = pandas.read_csv(data_path)
test_galaxies = set(data[data['cluster']=='Virgo']['galaxy'].unique())

In [2]:
df_train, dfs_test = select_galaxies(data, test_galaxies = test_galaxies)
df_train, dfs_test = replace_NaN(df_train, dfs_test)
df = create_data_dict(data, df_train, dfs_test)
df, _ = rescale_data(df)

input_dim = len(df['train']['inputs'][0])

Number of sources in training split after dropping rows with NaN as CI/m/color: 21232
535 sources have been dropped.
NaN values in testing data have been replaced with the correspnding median value observed in the training split


100%|█████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 3810.02it/s]

eccentricity will not be transformed. Skipped.
eccentricity_z will not be transformed. Skipped.


In [3]:
model = StandardClassifier(method='tree', params= {'criterion': 'entropy', 'min_samples_split':40, 'random_state': 4242422}, data=df)

Loading data...
Fitting model...


In [4]:
model.val()

/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [5]:
model.stats_validation

,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,0.907348,0.098413,0.041389,0.927965,0.966716,284,313,31,1062


In [6]:
model.test()

/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/dodo/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/dodo/git/globsML/globsML/utils/eval.py:146: RuntimeWarning: divide by zero encountered in double_scalars
  return auc(x,y)/(2*auc([0,x[-1]], [0,y[-1]])), x, y


In [7]:
model.stats_galaxies

,Galaxy,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,VCC1695,0.863636,0.344828,0.020450,0.717890,0.917782,19,22,10,511
1,VCC2092,0.820225,0.170455,0.050676,0.891018,0.941695,73,89,15,385
2,VCC538,0.900000,0.808511,0.066550,0.503202,0.961734,9,10,38,581
3,VCC731,0.871041,0.054054,0.085603,0.923029,0.952149,770,884,44,1398
4,VCC1528,0.875000,0.142857,0.024735,0.934800,0.980124,42,48,7,331
...,...,...,...,...,...,...,...,...,...,...
93,VCC1913,0.822581,0.163934,0.016667,0.842704,0.954610,51,62,10,662
94,VCC1993,1.000000,0.636364,0.028455,0.851795,0.995935,4,4,7,250
95,VCC543,0.923077,0.428571,0.028846,0.768694,0.965976,24,26,18,650
96,VCC1861,0.936170,0.214286,0.028571,0.917755,0.980319,44,47,12,467


In [8]:
model.stats_galaxies.mean()

/tmp/ipykernel_1841/3415850949.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  model.stats_galaxies.mean()


TPR               0.839113
FDR               0.257938
FPR               0.033374
AUC(FDR,TPR)           inf
AUC(FPR,TPR)      0.954558
# found GCs     111.112245
# total GCs     126.479592
# fake GCs       15.510204
# sources       644.510204
dtype: float64

In [9]:
model.stats_all

,Galaxy,TPR,FDR,FPR,"AUC(FDR,TPR)","AUC(FPR,TPR)",# found GCs,# total GCs,# fake GCs,# sources
0,ALL,0.878499,0.122492,0.029941,0.920554,0.967655,10889,12395,1520,63162
